In [1]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
result_dt = pd.read_excel('C:/Users/DELL/Desktop/result_dt.xlsx')
result_dt = result_dt.sort_values(by=0)

In [ ]:
data1=pd.read_table('C:/Users/DELL/Desktop/CASSimulation/allcasdataincsvformat10000loans.csv', sep = '|',index_col = 0)
#Projecting the UPB's for each loan for each of the 40 quarters
Monthly_Balance_Cas =[]
sumUPB=0
for index,row in data1.iterrows():
    sumUPB=sumUPB+ row['ORIGINAL UPB']
    Loan_Amount = row['ORIGINAL UPB']
    Interest_rate = row['CURRENT INTEREST RATE']
    
    Loan_Term = 360
    R = 1+(Interest_rate)/(12*100)
    X = Loan_Amount * (R**Loan_Term)*(1-R)/(1-R**Loan_Term)

    Monthly_Interest = []
    Monthly_Balance = []

    for i in range(1, 123):
        Interest = Loan_Amount*(R-1)
        Loan_Amount = (Loan_Amount - (X-Interest))
        if (i%3 ==0) and (i != 1):
            Monthly_Interest.append(Interest)
            Monthly_Balance.append(Loan_Amount)
            

    Monthly_Balance_Cas.append(Monthly_Balance)

In [ ]:
# Tranch Valuation by calculating the losses and payoffs for each of the loss scenarios
d1=100-3.92
d2=3.92-2.26
d3=2.26-1.73
d4=1.73-0.73
d5=0.73-0.00000000000000000000000000000000000000000000000000000001
loss_subordination=[0,0.73,1.73,2.26,3.92]
CDO_percentage=[d5,d4,d3,d2,d1]
referencepoolsize=sumUPB
tranchsize=[]
for i in CDO_percentage:
    tranchsize.append((referencepoolsize*i)/100)

In [ ]:
Tranche = pd.DataFrame()
Tranche['Loss Subordination'] = loss_subordination
Tranche['CDO %'] = CDO_percentage
Tranche['Tranche size'] = tranchsize
Tranche

,Loss Subordination,CDO %,Tranche size
0,0.00,0.73,1.828842e+07
1,0.73,1.00,2.505263e+07
2,1.73,0.53,1.327789e+07
3,2.26,1.66,4.158737e+07
4,3.92,96.08,2.407057e+09


In [ ]:
Tranch_Loss =[]*500
Tranch_Loss1=[]*500
Tranch_Loss2=[]*500
Tranch_Loss3=[]*500
Tranch_Loss4=[]*500
Tranch_Payoff = []
losses = [0]*500
payoff = [0]*500 
losses1 = [0]*500
payoff1 = [0]*500 
losses2=[0]*500
payoff2 = [0]*500 
losses3=[0]*500
payoff3 = [0]*500 
losses4=[0]*500
payoff4 = [0]*500 
#quarterly_rate = (((1.0254**(1/4))-1)*4)
for i in range(0,len(Tranche['Tranche size'])):
    
    payoffs =[]
    j=-1
    
    for el in result_dt[0]:
        j=j+1
        if(i==0):
            x = (min(Tranche['Tranche size'][i],(el*sumUPB/100)))
            losses[j]=x
            payoff[j]=Tranche['Tranche size'][i]-x
        elif(i==1):
            #print("hello")
            x=min(Tranche['Tranche size'][i],((el*sumUPB/100)-losses[j]))
            losses1[j]=x
            payoff1[j]=Tranche['Tranche size'][i]-x
            #print(x)
            #print(losses1[j])
        elif(i==2):
            x=min(Tranche['Tranche size'][i],((el*sumUPB/100)-losses[j]-losses1[j]))
            payoff2[j]=Tranche['Tranche size'][i]-x
            losses2[j]=x
        elif(i==3):
            x=min(Tranche['Tranche size'][i],((el*sumUPB/100)-losses[j]-losses1[j]-losses2[j]))
            payoff3[j]=Tranche['Tranche size'][i]-x
            losses3[j]=x
        else:
            x=min(Tranche['Tranche size'][i],((el*sumUPB/100)-losses[j]-losses1[j]-losses2[j]-losses3[j]))
            payoff4[j]=Tranche['Tranche size'][i]-x
            losses4[j]=x
        
        #payoffs.append(Tranche['Tranche size'][i]-x)
    
    if(i==0):
        Tranch_Loss.append(losses)
    elif(i==1):    
        Tranch_Loss1.append(losses1)
    elif(i==2):    
        Tranch_Loss2.append(losses2)
    elif(i==3):    
        Tranch_Loss3.append(losses3)
    else:
        Tranch_Loss4.append(losses4)
    
    #Tranch_Loss.append(losses)
    #Tranch_Payoff.append(payoffs)   

In [ ]:
Price1=0
Price2=0
Price3=0
Price4=0
Price5=0
payoff=[i*0.002 for i in payoff]
payoff1=[i*0.002 for i in payoff1]
payoff2=[i*0.002 for i in payoff2]
payoff3=[i*0.002 for i in payoff3]
payoff4=[i*0.002 for i in payoff4]

In [ ]:
for i in range(0,len(Tranche['Tranche size'])):
    for k in range(0,500):
        if(i==0):
            #print(payoff[1])
            #print(k)
            Price1=Price1+payoff[k]
            #print((1/500)*payoff[k])
        if(i==1):
            Price2=Price2+payoff1[k]
        if(i==2):
            Price3=Price3+payoff2[k]
        if(i==3):
            Price4=Price4+payoff3[k]
        if(i==4):
            Price5=Price5+payoff4[k]

In [ ]:
a=0.0254

In [ ]:
# Discounting the prices at moody's baseline 1 year rate
PriceofTranche1=Price1*math.exp(-a)
PriceofTranche2=Price2*math.exp(-a)
PriceofTranche3=Price3*math.exp(-a)
PriceofTranche4=Price4*math.exp(-a)
PriceofTranche5=Price5*math.exp(-a)

# Calculating the yield of B/BB and BBB/A rated tranche
Y1 = math.log(Tranche['Tranche size'][0]/PriceofTranche1)*100
Y2 = math.log(Tranche['Tranche size'][1]/PriceofTranche2)*100
Y3 = math.log(Tranche['Tranche size'][2]/PriceofTranche3)*100
Y4 = math.log(Tranche['Tranche size'][3]/PriceofTranche4)*100
Y5 = math.log(Tranche['Tranche size'][4]/PriceofTranche5)*100


Y = [Y1,Y2,Y3,Y4,Y5]
Prices = [PriceofTranche1,PriceofTranche2,PriceofTranche3,PriceofTranche4,PriceofTranche5]
Pricing = ['Tranche 1 no rating','Tranche 2 has no rating','B/BB','BBB/A','Tranche 5 has no rating']

Ratings = pd.DataFrame()
Ratings['Pricing'] = Pricing
Ratings['Prices'] = Prices
Ratings['Yield'] = Y

Ratings.set_index('Pricing')
Ratings.head()

,Pricing,Prices,Yield
0,Tranche 1 no rating,1.780765e+07,2.664001
1,Tranche 2 has no rating,2.442431e+07,2.540000
2,B/BB,1.294488e+07,2.540000
3,BBB/A,4.054435e+07,2.540000
4,Tranche 5 has no rating,2.346687e+09,2.540000
